### Using this notebook
The code is highly modularised, which means if you are interested in  directly viewing the results, you can skip most of the initial parts of the notebook and start from the section "baseline model".

### Improvements/ changes from the last attempt:
 - separate models for both the cities
 - smaller and flexible functions for data preprocesing to facilitate easy data reorganisation

#### importing the relevant packages and loading the data

In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split

from sklearn.metrics import mean_absolute_error
from sklearn.metrics import median_absolute_error
from sklearn.model_selection import ParameterGrid
from sklearn.model_selection import TimeSeriesSplit
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score
from sklearn.metrics import make_scorer

from scipy import stats

from xgboost import XGBRegressor

In [2]:
X = pd.read_csv("data/dengue_features_train.csv")
y = pd.read_csv("data/dengue_labels_train.csv")

#### Some helper functions for preprocessing

In [3]:
def impute(X):
    
    # remove the column that has ~20% null values, also ranks low on feature importance
    X.drop(['ndvi_ne'], axis=1, inplace=True)
    
    # Filling the rest using linear interpolation
    X.interpolate(inplace=True)

def remove_outliers(df):
    return df[(np.abs(stats.zscore(df)) < 5).all(axis=1)]

def mape(Y_test, Y_pred, epsilon = 1):
    return np.mean(np.abs((Y_test - Y_pred + epsilon) / (Y_test + epsilon))) * 100

def extract_month(s):
    return int(s[5:7])

def city_indices(X):
    # city boolean encoding
    return X.city == 'sj'

#### Preprocessing

In [4]:
def pre_process(X, trees = False):
    """
    Extracts the month out of date and converts it to a one hot
    Standardizes the numerical features
    """
    
    #Extracting month from the date
    months = X.week_start_date.apply(extract_month)

    # Removing the columns not required for classification
    X.drop(['city', 'year', 'weekofyear', 'week_start_date'], axis=1, inplace=True)

    # Standardizing the data
    if not trees:
        scaler = StandardScaler()
        X[X.columns] = scaler.fit_transform(X)

    # Month one hot features
    month_features = pd.get_dummies(months)
    X = X.join(month_features)

    # Alternatively use months as a discrete feature
    #X = X.join(months)
    
    return X

def seperate_cities_data(X, is_sj):
    """
    Separating the data for the two cities in order to create a different model for each
    """

    # Separating the cities data
    X_sj = X.loc[is_sj]
    X_iq = X.loc[~is_sj]
    
    return X_sj, X_iq

def get_y_labels(X_sj, X_iq, y):    
    
    y = y.total_cases    
    y_sj = y.loc[X_sj.index]
    y_iq = y.loc[X_iq.index]
    
    return y_sj, y_iq

def split(X_sj, X_iq, y_sj, y_iq):

    # train and test split
    sj_split_data = train_test_split(X_sj, y_sj, shuffle = False)
    iq_split_data = train_test_split(X_iq, y_iq, shuffle = False)

    return sj_split_data, iq_split_data

def process(X, y = pd.Series(), train = True, trees = False):
    """
    preprocesses the data
    
    arguements:
    X - the data to be pre processed
    y - labels, optional
    train - if False, computes pre processing for test data set.
    tress - True if the model to be used is a tree based one, for eg. XGBoost
    """
    
    is_sj = city_indices(X)
    if not trees:
        impute(X)
    X = pre_process(X, trees)
    
    #X = remove_outliers(X)
    X_sj, X_iq = seperate_cities_data(X, is_sj)
    if y.empty:
        return X_sj, X_iq
    
    y_sj, y_iq = get_y_labels(X_sj, X_iq, y)
    if not train:
        return X_sj, X_iq, y_sj, y_iq
    
    return split(X_sj, X_iq, y_sj, y_iq)

#### Random Model

In [5]:
data = process(X,y)
(X_sj_train, X_sj_test, Y_sj_train, Y_sj_test), (X_iq_train, X_iq_test, Y_iq_train, Y_iq_test) = data

In [6]:
def random(Y_test, Y_train):
    y_p = np.full(len(Y_test), np.mean(Y_train))
    return mean_absolute_error(Y_test, y_p)

> The performance on random models shall guide us on improving our models

In [7]:
random(Y_sj_test, Y_sj_train)

28.027284681130833

In [8]:
random(Y_iq_test, Y_iq_train)

8.287337278106508

In [9]:
Y_test = Y_sj_test.append(Y_iq_test)
Y_train = Y_sj_train.append(Y_iq_train)

In [10]:
random(Y_test, Y_train)

20.337127158555734

#### Generalised Model
A generalised model is nothing but the combination of accuracy scores on different cities trained on different models!

In [11]:
def general_model(clf, data):
    """
    returns the mean absolute error combined for both the cities.
    """
    (X_sj_train, X_sj_test, Y_sj_train, Y_sj_test), (X_iq_train, X_iq_test, Y_iq_train, Y_iq_test) = data
    
    clf.fit(X_sj_train, Y_sj_train, eval_metric = mean_absolute_error)
    Y_sj_pred = clf.predict(X_sj_test)
    
    clf.fit(X_iq_train, Y_iq_train, eval_metric = mean_absolute_error)
    Y_iq_pred = clf.predict(X_iq_test)
    
    Y_pred = np.concatenate([Y_sj_pred, Y_iq_pred])
    Y_pred = Y_pred.astype(int).clip(0)
    Y_test = Y_sj_test.append(Y_iq_test)
    
    return mean_absolute_error(Y_test, Y_pred)

#### Baseline Model

In [12]:
X = pd.read_csv("data/dengue_features_train.csv")
y = pd.read_csv("data/dengue_labels_train.csv")

In [13]:
data = process(X,y, trees=True)

In [14]:
model = XGBRegressor()
general_model(model, data)

18.26923076923077

> A simple XGBoost regressor gives slight improvement on the baseline but nothing significant

#### tuned model

In [15]:
# we find gamma=0 and max_depth=3, defaut parameters to be best, rest tuned parameters can be seen below
rf = XGBRegressor(n_estimators=55, learning_rate=0.01 ,n_jobs=-1, subsample=0.9, colsample_bytree=0.6, colsample_bylevel=0.1, min_child_weight=5, reg_alpha=0.1) 

general_model(rf, data)

12.39010989010989

> A tuned XGB model shows some significant improvement

#### cross validation scores
To examine things more closely, let us view the scores on cross-validations, we should ideally use time-based cross-validations, since we do not have enough data, let us for once just go for the regular thing!

In [16]:
rf = XGBRegressor(n_estimators=55, learning_rate=0.01 ,n_jobs=-1, subsample=0.9, colsample_bytree=0.6, colsample_bylevel=0.1, min_child_weight=5, reg_alpha=0.1) 
(X_sj_train, X_sj_test, Y_sj_train, Y_sj_test), (X_iq_train, X_iq_test, Y_iq_train, Y_iq_test) = data
cross_val_score(rf, X_sj_train, Y_sj_train, cv = 5, scoring=make_scorer(mean_absolute_error))

array([30.59434104, 49.11759507, 14.95902743, 32.26011295, 12.66011701])

In [17]:
cross_val_score(rf, X_iq_train, Y_iq_train, cv = 5, scoring=make_scorer(mean_absolute_error))

array([3.25106094, 7.87761068, 7.00398358, 5.8487702 , 3.53553   ])

> There is too much variation across cross-validations, we need to rethink our modelling approches

Let us also have a look at **time-based cross-validations**

In [18]:
clf = XGBRegressor(n_estimators=55, learning_rate=0.01 ,n_jobs=-1, subsample=0.9, colsample_bytree=0.6, colsample_bylevel=0.1, min_child_weight=5, reg_alpha=0.1)

In [19]:
cv_splits = TimeSeriesSplit(n_splits=3).split(X_sj_train, Y_sj_train)

cross_val_score(clf, X_sj_train, Y_sj_train, cv=cv_splits, scoring=make_scorer(mean_absolute_error))

array([40.78982837, 31.14011517, 12.58860924])

In [20]:
cv_splits = TimeSeriesSplit(n_splits=3).split(X_iq_train, Y_iq_train)
cross_val_score(clf, X_iq_train, Y_iq_train, cv=cv_splits, scoring=make_scorer(mean_absolute_error))

array([5.49918169, 8.7481604 , 4.26543863])

#### Conclusion : 
XGBoost does perform good on a tuned model but a closer examination of cross-validation scores reveals a huge variance in the results, the improvement in the accuracy of the tuned model is likely the result of overfitting to one train-test distribution.
 
#### Next Steps :
We are going to need to take a closer look at the distribution of our data and based on that choose some more advanced statistical models, perhaps outside of the standard libraries like the scikit learn